In [1]:
import pandas as pd
import numpy as np 

In [ ]:
from google.colab import files
uploaded = files.upload() # Choose the excel file from your device 

In [ ]:
import re
lat = '''25°1'18"N'''
deg, minutes, seconds, direction =  re.split('[°\'"]', lat)
(float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1) 

25.021666666666665

In [ ]:
import io
mining_df = pd.read_excel(io.BytesIO(uploaded['DMS_to_LatLong.xlsx']))
mining_df.head(20)

In [ ]:
def get_lat_lon(deg, minutes, seconds):
    return (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1) # Calculate the longitude and the latitude

In [ ]:
df = mining_df[mining_df['Y'].notna()] # Delete the rows with missing values
df.head(20)

In [ ]:
df= df[pd.to_numeric(df['D'], errors='coerce').notnull()] # Delete the nonnumeric rows
df.head(20)

,Y,X,D,M,S,D.1,M.1,S.1,P
0,2769254.392,728323.36,25,1,18,47,15,45.6,1
1,2779036.337,748997.019,25,6,24,47,28,9,2
2,2775781.614,752813.833,25,4,36,47,30,23,3
3,2772356.603,747497.487,25,2,47.92,47,27,11.18,4
4,2767208.788,742530.091,25,0,3.6,47,24,10.8,5
5,2763395.255,736766.372,24,58,3,47,20,43,6
6,2763799.272,731225.956,24,58,19.2,47,17,25.8,7
7,2763188.826,729889.362,24,58,0.1,47,16,37.8,8
10,2817176.088,693848.141,25,27,32.4,46,55,40.8,1
11,2805080.951,714562.92,25,20,49.2,47,7,55.2,2


In [ ]:
df['lat'] = df.apply(lambda x: get_lat_lon(deg = x['D'], minutes = x[' M'], seconds = x['S']), axis=1) # Create the latitude column

<ipython-input-11-7cf97c548761>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat'] = df.apply(lambda x: get_lat_lon(deg = x['D'], minutes = x[' M'], seconds = x['S']), axis=1)


In [ ]:
df['lon'] = df.apply(lambda x: get_lat_lon(deg = x['D.1'], minutes = x[' M.1'], seconds = x['S.1']), axis=1) # Create the longitude colomn

<ipython-input-12-22fbce49b363>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lon'] = df.apply(lambda x: get_lat_lon(deg = x['D.1'], minutes = x[' M.1'], seconds = x['S.1']), axis=1)


In [ ]:
!pip install simplekml  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65874 sha256=6ac58ac48fa60e4fd179872b1e5aa458bb2b3f4bf60c4346111dfb39719eb53e
  Stored in directory: /root/.cache/pip/wheels/64/ed/fa/ce203ffaab4888a5365e9b2298b50f93705391e25098fd7a3d
Successfully built simplekml


In [ ]:
import simplekml
import pprint
import numpy as np

kml = simplekml.Kml()

In [ ]:
df_modify=df[['lon', 'lat', 'P']] 

In [ ]:
df_modify['P']=df_modify['P'].apply(pd.to_numeric, errors='coerce') # Convert the type from object to numeric

<ipython-input-16-f28633dd6701>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modify['P']=df_modify['P'].apply(pd.to_numeric, errors='coerce')


In [ ]:
# NOTE: To create the kml file, we must give the same name to every point that is a part of the same polygon.
i=0
df_modify['polygon']=0
for x in range(len(df_modify)):
  if df_modify['P'].iloc[x]==1:
    i+=1
    df_modify['polygon'].iloc[x]=i

  else:
    df_modify['polygon'].iloc[x]=i

<ipython-input-17-81fd2dbaa3e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modify['polygon']=0
<ipython-input-17-81fd2dbaa3e7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modify['polygon'].iloc[x]=i
<ipython-input-17-81fd2dbaa3e7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modify['polygon'].iloc[x]=i


In [ ]:
dict_dataframes=dict(tuple(df_modify.groupby('polygon'))) # Create a dictionary of dataframes, each containing a single polygon.

{1:          lon        lat  P  polygon
 0  47.262667  25.021667  1        1
 1  47.469167  25.106667  2        1
 2  47.506389  25.076667  3        1
 3  47.453106  25.046644  4        1
 4  47.403000  25.001000  5        1
 5  47.345278  24.967500  6        1
 6  47.290500  24.972000  7        1
 7  47.277167  24.966694  8        1, 2:        lon     lat  P  polygon
 10  46.928  25.459  1        2
 11  47.132  25.347  2        2
 12  47.102  25.196  3        2
 13  46.872  25.208  4        2, 3:           lon        lat  P  polygon
 16  45.541111  25.974444  1        3
 17  45.590000  25.984167  2        3
 18  45.681389  25.866111  3        3
 19  45.632778  25.853333  4        3
 20  45.585594  25.915131  5        3
 21  45.581075  25.913392  6        3
 22  45.579042  25.917869  7        3
 23  45.582853  25.919411  8        3, 4:        lon     lat  P  polygon
 25  43.353  25.066  1        4}

In [ ]:
for polygon, df in dict_dataframes.items():

    # Create a List of tuples with the columns of the dataframe
    tuples = [tuple(x) for x in df.values]

    extrude=1
    altitudemode = simplekml.AltitudeMode.relativetoground

    pol = kml.newpolygon(name = str(polygon), description="District of " + str(polygon), outerboundaryis=tuples, extrude=extrude, altitudemode=altitudemode)
    pol.style.linestyle.color = simplekml.Color.honeydew
    pol.style.linestyle.width = 3
    pol.style.polystyle.color = simplekml.Color.changealphaint(100, simplekml.Color.navy)

   
kml.save('C:\\Users\map.kml') #Save the Kml file 

In [ ]:
lon = df_modify['lon']
lat = df_modify['lat']

In [ ]:
unique=df_modify['polygon'].unique()
unique

array([1, 2, 3, 4])

In [ ]:
Polygon={}

In [ ]:
'''
NOTE: In order to be able to use the polygon area calculation function("p_area"), 
the coordinates must be in this form [[[longitude,latitude ],[longitude,latitude ]... ]] for each polygon
'''
for key in unique:
  Polygon[key]=[[]]
  for i in range(len(df_modify)):
    if df_modify['polygon'].iloc[i]==key:
     Polygon[key][0].append([lon.iloc[i],lat.iloc[i]])

In [ ]:
Polygon

{1: [[[47.26266666666667, 25.021666666666665],
   [47.469166666666666, 25.10666666666667],
   [47.506388888888885, 25.076666666666668],
   [47.45310555555556, 25.046644444444446],
   [47.403, 25.001],
   [47.34527777777778, 24.967499999999998],
   [47.2905, 24.971999999999998],
   [47.277166666666666, 24.966694444444443]]],
 2: [[[46.928, 25.459],
   [47.132, 25.346999999999998],
   [47.102000000000004, 25.196],
   [46.872, 25.208]]],
 3: [[[45.54111111111111, 25.974444444444444],
   [45.59, 25.984166666666667],
   [45.68138888888889, 25.866111111111113],
   [45.632777777777775, 25.853333333333335],
   [45.585594444444446, 25.915130555555553],
   [45.581075000000006, 25.913391666666666],
   [45.57904166666667, 25.917869444444445],
   [45.58285277777778, 25.919411111111113]]],
 4: [[[43.353, 25.066]]]}

In [2]:
!pip install area

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for area: filename=area-1.1.1-py3-none-any.whl size=3622 sha256=288237fea2b7b5019c46e46f133684e5e34ab46d4dec9a5d01b7a796f561ef6a
  Stored in directory: /root/.cache/pip/wheels/4f/99/e9/0a659ec7bb7f9b457c6a1cf77cff5b34de511e1f5d2236fbab
Successfully built area


In [ ]:
# Calculate the polygon area in km2.
from area import area
def p_area(coordinates):
  obj = {'type':'Polygon','coordinates': coordinates}
  return area(obj)/10**6

In [ ]:
Polygons_area=pd.DataFrame(columns = ["area"])

In [ ]:
Polygons_area["area"]=[p_area(i) for i in Polygon.values()]

In [ ]:
Polygons_area 

,area
0,171.138799
1,518.465579
2,76.812767
3,0.000000
